In [3]:
import pandas as pd

df = pd.read_csv("../data/spotify-tracks-ids.csv")
df

,spotify_id,artist_name,track_name,class
0,4bk6zQvr1tEokVjLJ7xZfz,Squid,Paddling,6.0
1,2A110W6j3xAbTDsOXu4uN0,Kokomo Arnold,Mean Old Twister - Remastered,2.0
2,7vFaliU9GLPLNwyu2Kg0YT,Brynn Elliott,Letter To A Girl,9.0
3,5jHL8tcDLiiXXKcGr4nkWk,LASTELLE,A Letter Unread,8.0
4,1oht5GevPN9t1T3kG1m1GO,James Taylor,Fire and Rain,0.0
...,...,...,...,...
17759,0XBjcu4H7ZHD7nZykpX2cN,Movement At Night,No Matter What,10.0
17760,3A2OHirUAlQp2x73BJlEDA,Mike WiLL Made-It,"Perfect Pint (feat. Kendrick Lamar, Gucci Mane...",5.0
17761,6qXBZGMoNHTVIebQDECoY9,The Maine,Sticky,10.0
17762,68Cz5sfSKx1ugwoxJzLIHh,Kelpe,All the Way Round - Edit,1.0


In [4]:
df.drop_duplicates(["artist_name", "track_name", "class"], keep="first", inplace=True)
df

,spotify_id,artist_name,track_name,class
0,4bk6zQvr1tEokVjLJ7xZfz,Squid,Paddling,6.0
1,2A110W6j3xAbTDsOXu4uN0,Kokomo Arnold,Mean Old Twister - Remastered,2.0
2,7vFaliU9GLPLNwyu2Kg0YT,Brynn Elliott,Letter To A Girl,9.0
3,5jHL8tcDLiiXXKcGr4nkWk,LASTELLE,A Letter Unread,8.0
4,1oht5GevPN9t1T3kG1m1GO,James Taylor,Fire and Rain,0.0
...,...,...,...,...
17759,0XBjcu4H7ZHD7nZykpX2cN,Movement At Night,No Matter What,10.0
17760,3A2OHirUAlQp2x73BJlEDA,Mike WiLL Made-It,"Perfect Pint (feat. Kendrick Lamar, Gucci Mane...",5.0
17761,6qXBZGMoNHTVIebQDECoY9,The Maine,Sticky,10.0
17762,68Cz5sfSKx1ugwoxJzLIHh,Kelpe,All the Way Round - Edit,1.0


In [5]:
df_fitered = df[df["spotify_id"].notna()]
df_fitered

,spotify_id,artist_name,track_name,class
0,4bk6zQvr1tEokVjLJ7xZfz,Squid,Paddling,6.0
1,2A110W6j3xAbTDsOXu4uN0,Kokomo Arnold,Mean Old Twister - Remastered,2.0
2,7vFaliU9GLPLNwyu2Kg0YT,Brynn Elliott,Letter To A Girl,9.0
3,5jHL8tcDLiiXXKcGr4nkWk,LASTELLE,A Letter Unread,8.0
4,1oht5GevPN9t1T3kG1m1GO,James Taylor,Fire and Rain,0.0
...,...,...,...,...
17759,0XBjcu4H7ZHD7nZykpX2cN,Movement At Night,No Matter What,10.0
17760,3A2OHirUAlQp2x73BJlEDA,Mike WiLL Made-It,"Perfect Pint (feat. Kendrick Lamar, Gucci Mane...",5.0
17761,6qXBZGMoNHTVIebQDECoY9,The Maine,Sticky,10.0
17762,68Cz5sfSKx1ugwoxJzLIHh,Kelpe,All the Way Round - Edit,1.0


In [6]:
# importing the sys module
import sys        
 
# appending the directory of mod.py
# in the sys.path list
sys.path.append('../service/spotify_service.py')       

In [7]:
# importing the sys module
import sys        
 
# appending the directory of mod.py
# in the sys.path list
sys.path.append('/d/')   

from service.spotify_service import SpotifyService

spotify_service = SpotifyService()
batch_size = 100
n_rows, n_columns = df_fitered.shape
batches_n = n_rows // batch_size
df_filtered_values = df_fitered.values

audio_features_dict = {
    "acousticness": [],
    "analysis_url": [],
    "danceability": [],
    "duration_ms": [],
    "energy": [],
    "spotify_id": [],
    "instrumentalness": [],
    "key": [],
    "liveness": [],
    "loudness": [],
    "mode": [],
    "speechiness": [],
    "tempo": [],
    "time_signature": [],
    "valence": []
}

for i in range(batches_n):
    start_index = i * batch_size
    end_index = start_index + batch_size
    df_batch = df_filtered_values[start_index:end_index]
    audio_features_batch_dict = spotify_service.get_spotify_tacks_audio_features(
        df_batch
    )
    for k in audio_features_batch_dict:
        if audio_features_dict.get(k) == None:
            audio_features_dict[k] = audio_features_batch_dict[k]
        else:
            audio_features_dict[k] += audio_features_batch_dict[k]

if n_rows % batch_size != 0:
    start_index = batches_n * batch_size
    df_batch = df_filtered_values[start_index:n_rows]
    audio_features_batch_dict = spotify_service.get_spotify_tacks_audio_features(
        df_batch
    )
    for k in audio_features_batch_dict:
        if audio_features_dict.get(k) == None:
            audio_features_dict[k] = audio_features_batch_dict[k]
        else:
            audio_features_dict[k] += audio_features_batch_dict[k]

df_audio_features = pd.DataFrame(audio_features_dict)
df_audio_features

KeyboardInterrupt: 

In [ ]:
from pathlib import Path

df_merged = df_audio_features.merge(
    df_fitered, left_on="spotify_id", right_on="spotify_id"
)
filepath = Path("../data/spotify-tracks-audio-features.csv")
filepath.parent.mkdir(parents=True, exist_ok=True)
df_merged.to_csv(filepath, index=False)